In [1]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

In [17]:
class BackupApiResp:
    def __init__(self,path_file_bk, max_days_to_valid_data = 720): 
        self.__con = sqlite3.connect(path_file_bk)
        self.__con.row_factory = sqlite3.Row
        self.days_to_expires = max_days_to_valid_data #validade dos dados
        self.today = datetime.now().strftime("%Y-%m-%d %H:%M")
        self.__init_db()
    
    def __init_db(self):
        sql_table_def = (
            'CREATE TABLE IF NOT EXISTS ' 
            'anuncios_resumo ('
                'url_lista TEXT,'
                'categoria_completa TEXT,'
                'categoria_atual TEXT,'
                'localizacao_completa TEXT,'
                'data_coleta_dados DATETIME,'
                'titulo_anuncio TEXT,'
                'preco_anuncio FLOAT,'
                'anuncio_profissional INTEGER,'
                'url_anuncio TEXT,'
                'data_publicacao_anuncio DATETIME'
            ');'
        )

        sql_index1_def = (
            'CREATE INDEX IF NOT EXISTS '
            'index_url_anuncio_ar ON '
            'anuncios_resumo ('
                'url_anuncio ASC'
            ');'
        )

        sql_index2_def = (
            'CREATE INDEX IF NOT EXISTS '
            'index_url_lista_ar ON '
            'anuncios_resumo ('
                'url_lista ASC'
            ');'
        )

        self.__con.execute(sql_table_def)
        self.__con.execute(sql_index1_def)
        self.__con.execute(sql_index2_def)
        self.__con.commit()
    
    # def __del_row(self,cnpj,source,query=None):
        
    #     query_where_str = ""
    #     if query:
    #         _query = str(query).replace("'","''")
    #         query_where_str = f"AND query = '{_query}'"

    #     sql_del_row = (
    #         "DELETE FROM backup WHERE "
    #         f"cnpj = '{cnpj}' and source = '{source}' {query_where_str};"
    #     )
    #     self.__con.execute(sql_del_row)

    def add_row_anuncios_resumo(self,url_lista,categoria_completa,categoria_atual,localizacao_completa,titulo_anuncio,preco_anuncio, anuncio_profissional,url_anuncio,data_publicacao_anuncio):
        sql_insert_data = (
            "INSERT INTO anuncios_resumo (url_lista,categoria_completa,categoria_atual,localizacao_completa,data_coleta_dados,titulo_anuncio,preco_anuncio, anuncio_profissional,url_anuncio,data_publicacao_anuncio) "
            "VALUES ("
                f"'{url_lista}',"
                f"'{categoria_completa}',"
                f"'{categoria_atual}',"
                f"'{localizacao_completa}',"
                f"'{self.today}',"
                f"'{titulo_anuncio}',"
                f"{preco_anuncio},"
                f"{anuncio_profissional},"
                f"'{url_anuncio}',"
                f"'{data_publicacao_anuncio}' "
            ");"
        )
        self.__con.execute(sql_insert_data)
        self.__con.commit()
    
    def date_is_valid(self,date:str):
        data_age_days = (datetime.now()-datetime.fromisoformat(date)).days
        return (data_age_days <= self.days_to_expires)

    def has_url_ad_in_anuncio_resumo(self,url_ad):
        sql = f""" select url_anuncio from anuncios_resumo
                    where url_anuncio = '{url_ad}'
                    limit 1 """

        result = self.__con.execute(sql).fetchone()
        return True if result else False







In [12]:
def convert_date_olx_to_datetime_str(texto_data):
    MONTHS = {'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4,  'mai': 5,  'jun': 6,
          'jul': 7, 'ago': 8, 'set': 9, 'out': 10, 'nov': 11, 'dez': 12}
    data, hora = texto_data.lower().split(',')
    data_padrao = ""
    
    if 'ontem' == data :
        data_padrao = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
    elif 'hoje' == data :
        data_padrao = datetime.now().strftime('%Y-%m-%d')
    else:    
        dataP = data.split(' ')
        dia_mes = int(dataP[0][0:2])
        mes = MONTHS[dataP[1]]
        ano_atual = int(datetime.now().strftime('%Y'))
        mes_atual = int(datetime.now().strftime('%m'))
        ano = ano_atual if mes_atual >= mes else ano_atual - 1
        data_padrao = datetime(year=ano, month=mes, day=dia_mes).strftime('%Y-%m-%d')
    
    return data_padrao + hora

In [18]:
backup = BackupApiResp("./banco_backup.db")

In [21]:
#Teste
backup.add_row_anuncios_resumo('www.teste','hobbys;livros','livros','Brasil;Minas;31','mágico de oz',200.98, 1,'www.teste2','2022-08-30 15:21')
backup.has_url_ad_in_anuncio_resumo('www.teste2')

True